In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import Utils.evaluation
import Utils.model_utils
import torch

In [ ]:
import torchaudio
import torch
from pathlib import Path

denoise_model = torch.load("denoise_checkpoint.pth", map_location=torch.device("cpu"))

model_params = denoise_model["model_params"]

model = Utils.model_utils.DenoiseNet(
    num_features=model_params["num_features"],
    num_segments=model_params["num_segments"]
)
normalization_values = denoise_model["normalization"]

model.load_state_dict(denoise_model["model_state"])
model.eval()

In [10]:
from IPython.display import Audio

noisy_folder = "../Audios/Augmented Audios 3 SNR"
noisy_audios = list(Path(noisy_folder).rglob('*.flac'))
noisy_input = torchaudio.load(noisy_audios[3])

denoised_output = Utils.evaluation.denoise_audio(model, noisy_input[0], model_params, normalization_values["noisy_mean"], normalization_values["noisy_std"], normalization_values["clean_mean"], normalization_values["clean_std"])

torchaudio.save("noisy.wav", noisy_input[0].cpu(), 16000)
torchaudio.save("denoised.wav", denoised_output.unsqueeze(0).cpu(), 16000)

In [11]:

# Denoised
Audio(denoised_output.cpu().numpy(), rate=16000)

In [12]:

# Noisy
Audio(noisy_input[0].cpu().numpy(), rate=16000)

In [ ]:
from IPython.display import Audio

import torch

# Cargar audios limpio y denoised (ya deben estar alineados y con misma fs)
clean_audio, sr = torchaudio.load("clean.wav")
denoised_audio, _ = torchaudio.load("denoised.wav")

snr_metric = SignalNoiseRatio()
snr_value = snr_metric(denoised_audio, clean_audio)
print(f"SNR: {snr_value:.2f} dB")

pesq_score = pesq(sr, clean_audio.squeeze().numpy(), denoised_audio.squeeze().numpy(), 'wb')
print(f"PESQ: {pesq_score:.3f}")
# STOI
stoi_score = stoi(clean_audio.squeeze().numpy(), denoised_audio.squeeze().numpy(), sr, extended=False)
print(f"STOI: {stoi_score:.3f}")
